In [117]:
import gensim
import json
import logging
import re
import numpy as np

logging.basicConfig(format='%(levelname)s : %(message)s', level=logging.INFO)
logging.root.level = logging.INFO  # ipython sometimes messes up the logging setup; restore

from nltk import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from pymystem3 import Mystem

STEMMER = Mystem()

STOPWORDS_FILE = 'data/russian_stopwords.txt'
with open(STOPWORDS_FILE, encoding='utf-8') as sf:
    STOPWORDS = {line.strip() for line in sf}

STOPWORDS.update(["1", "2", "3", "4", "5", "6", "7", "8", "9", "0", "http", "vk", "cc", "com"])

DATA_FILE = 'data/text_posts.json'
with open(DATA_FILE, encoding='utf-8') as df:
    TEXTS = json.loads(df.read())
    
N_TOPICS = 5

# Препроцессинг данных


Определяем функции для токенизации

In [125]:
def preprocess(text, tokenize = RegexpTokenizer(r'\w+').tokenize, stopwords=STOPWORDS, stem=STEMMER.lemmatize):
    # TODO: add lemmatizer
    text = re.sub(r'^https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)
    lowercase_words = [token.lower() for token in tokenize(text)]
    no_stopwords = [word for word in lowercase_words if word not in stopwords]
    stemmed = [stem(word)[0] for word in no_stopwords]
    return list(w for w in stemmed if w not in stopwords and not w.isdigit() and len(w) > 2)
preprocess(u'Я и нe cтрeмлюcь')

['стремиться']

In [126]:
tokens4texts = [preprocess(text) for text in TEXTS] # текст -> массив слов

In [127]:
id2word = gensim.corpora.Dictionary(tokens4texts)
id2word.filter_extremes(no_below=20, no_above=0.1)
print(id2word)

INFO:gensim.corpora.dictionary:adding document #0 to Dictionary(0 unique tokens: [])
INFO:gensim.corpora.dictionary:adding document #10000 to Dictionary(8425 unique tokens: ['стук', 'подпускать', 'бмвшник', 'молчать', 'судзумие']...)
INFO:gensim.corpora.dictionary:adding document #20000 to Dictionary(10830 unique tokens: ['стук', 'подпускать', 'бмвшник', 'молчать', 'искрить']...)
INFO:gensim.corpora.dictionary:built Dictionary(10897 unique tokens: ['стук', 'подпускать', 'бмвшник', 'молчать', 'искрить']...) from 20407 documents (total 146525 corpus positions)
INFO:gensim.corpora.dictionary:discarding 9690 tokens: [('автор', 5), ('античный', 1), ('блестеть', 11), ('воплощать', 1), ('вымолвить', 1), ('глубина', 8), ('заветный', 7), ('замирать', 10), ('кубик', 4), ('легкий', 17)]...
INFO:gensim.corpora.dictionary:keeping 1207 tokens which were in no less than 20 and no more than 2040 (=10.0%) documents
INFO:gensim.corpora.dictionary:resulting dictionary: Dictionary(1207 unique tokens: ['бе

Dictionary(1207 unique tokens: ['берет', 'точно', 'голова', 'фото', 'молчать']...)


Преобразуем тектсы в вектора:

In [128]:
corpus = [id2word.doc2bow(doc) for doc in tokens4texts]

# Обучение

In [129]:
clipped_corpus = gensim.utils.ClippedCorpus(corpus, 10000)

In [130]:
%time lda_model = gensim.models.LdaModel(clipped_corpus, num_topics=10, id2word=id2word, passes=4)

INFO:gensim.models.ldamodel:using symmetric alpha at 0.1
INFO:gensim.models.ldamodel:using serial LDA version on this node
INFO:gensim.models.ldamodel:running online LDA training, 10 topics, 4 passes over the supplied corpus of 10000 documents, updating model once every 2000 documents, evaluating perplexity every 10000 documents, iterating 50x with a convergence threshold of 0.001000
INFO:gensim.models.ldamodel:PROGRESS: pass 0, at document #2000/10000
INFO:gensim.models.ldamodel:merging changes from 2000 documents into a model of 10000 documents
INFO:gensim.models.ldamodel:topic #0 (0.100): 0.035*счастие + 0.030*женщина + 0.018*нужный + 0.017*хотеться + 0.016*девушка + 0.013*мужчина + 0.013*говорить + 0.011*счастливый + 0.011*знать + 0.009*смотреть
INFO:gensim.models.ldamodel:topic #1 (0.100): 0.023*самый + 0.018*девушка + 0.018*работа + 0.011*мужчина + 0.011*простой + 0.009*понимать + 0.009*шаг + 0.009*женщина + 0.008*месяц + 0.008*давать
INFO:gensim.models.ldamodel:topic #2 (0.100):

CPU times: user 1min 6s, sys: 386 ms, total: 1min 7s
Wall time: 1min 7s


In [131]:
%time lsi_model = gensim.models.LsiModel(clipped_corpus, id2word=id2word, num_topics=5)

INFO:gensim.models.lsimodel:using serial LSI version on this node
INFO:gensim.models.lsimodel:updating model with new documents
INFO:gensim.models.lsimodel:preparing a new chunk of documents
INFO:gensim.models.lsimodel:using 100 extra samples and 2 power iterations
INFO:gensim.models.lsimodel:1st phase: constructing (1207, 105) action matrix
INFO:gensim.models.lsimodel:orthonormalizing (1207, 105) action matrix
INFO:gensim.models.lsimodel:2nd phase: running dense svd on (105, 10000) matrix
INFO:gensim.models.lsimodel:computing the final decomposition
INFO:gensim.models.lsimodel:keeping 5 factors (discarding 74.807% of energy spectrum)
INFO:gensim.models.lsimodel:processed documents up to #10000
INFO:gensim.models.lsimodel:topic #0(61.099): 0.747*"любовь" + 0.224*"хороший" + 0.208*"женщина" + 0.168*"самый" + 0.144*"мужчина" + 0.139*"друг" + 0.127*"девушка" + 0.119*"история" + 0.101*"сердце" + 0.098*"ночь"
INFO:gensim.models.lsimodel:topic #1(47.823): -0.588*"говорить" + 0.381*"любовь" +

CPU times: user 1.44 s, sys: 92.5 ms, total: 1.54 s
Wall time: 1.27 s


In [132]:
def print_top_terms(lda, num_terms=10):
    for i in range(0, lda.num_topics):
        terms = [term for val, term in lda.show_topic(i, num_terms)]
        print("topic #", str(i), ": ", " ".join(terms))  

In [134]:
print_top_terms(lda_model)

topic # 0 :  счастие говорить хотеться свой делать больно увидеть никто показывать смотреть
topic # 1 :  девушка парень бог встреча добавлять стена сердце шаг либо давать
topic # 2 :  жить знать равно душа сильно ждать маленький звонить позвонить плохо
topic # 3 :  самый красивый любимый ценить хороший понимать счастливый ребенок видеть добрый
topic # 4 :  находить нравиться голова вопрос знать отвечать взгляд момент слышать верить
topic # 5 :  должный потерять мужчина влюбляться глаз сильный единственный забирать важно подруга
topic # 6 :  мужчина женщина настоящий мир плакать спать ребенок утро знать слово
topic # 7 :  любовь друг дружба хороший девочка посмотреть поцелуй нога мальчик жена
topic # 8 :  хороший думать нужный говорить давать бояться забывать знать понимать случаться
topic # 9 :  мама прощать поступок ценить видно мечтать самый звонок дорога мечта


In [135]:
print_top_terms(lsi_model)

topic # 0 :  любовь хороший женщина самый мужчина друг девушка история сердце ночь
topic # 1 :  говорить любовь знать мужчина грудь самый бояться становиться рука понимать
topic # 2 :  говорить мужчина грудь самый любовь женщина бояться становиться месяц рука
topic # 3 :  нога рука грудь мужчина женщина пол сторона прямой секунда месяц
topic # 4 :  мужчина женщина нога рука самый пол сторона работа прямой секунда


# Оценка модели

In [122]:
def intra_inter(model, test_docs, vectorize=id2word.doc2bow, num_pairs=10000):
    # split each test document into two halves and compute topics for each half
    part1 = [model[vectorize(tokens[: len(tokens) // 2])] for tokens in test_docs]
    part2 = [model[vectorize(tokens[len(tokens) // 2 :])] for tokens in test_docs]
    
    # print computed similarities (uses cossim)
    print("average cosine similarity between corresponding parts (higher is better):")
    print(np.mean([gensim.matutils.cossim(p1, p2) for p1, p2 in zip(part1, part2)]))

    random_pairs = np.random.randint(0, len(test_docs), size=(num_pairs, 2))
    print("average cosine similarity between 10,000 random parts (lower is better):")    
    print(np.mean([gensim.matutils.cossim(part1[i[0]], part2[i[1]]) for i in random_pairs]))

In [137]:
intra_inter(model=lda_model, test_docs=tokens4texts[10000:11000])

IndexError: index 1219 is out of bounds for axis 1 with size 1207

In [138]:
intra_inter(model=lsi_model, test_docs=tokens4texts[10000:11000])

average cosine similarity between corresponding parts (higher is better):
0.474112356861
average cosine similarity between 10,000 random parts (lower is better):
0.441618392044
